# ROI pull

## Step 1: Prepare Data  

## Step 2: Transform in asymmetrical space with `flirt`    
**`flirt`**:  used to apply a saved transformation to a volume (`--applyxfm`, `-init`, `-out`)
-- note for this the reference volume must be specified, it sets the voxel and image dimensions of the resulting volume.

## Step 3: Threshold fMRI activations  `fslmaths -thr` 

## Step 4: Calculate ROI regions for each individual with `fslmeants`
`fslmeants: outputs the average of a timeseries.`  
Used fslmeants to calculate volumes for that cluster on each individual, 
extracting ROI regions, by subject by condition. 

## Step 5: Combine each ROI clusters into a matrix, by subject per condition
## Step 6: Build functional connectivity subject/condition matrices from the ROI subject/condition matrices
## Step 7: Build functional connectivity subject matrices from averages of the conditions by subject

In [ ]:
#########################################
## Step 1:

# set data variables

print("[INFO] PROGRAM STARTING....")
# path that holds the subjects
data_path='/projects/niblab/experiments/project_milkshake/derivatives'

# get subject task functional images (niftis)
# -- note: this code grabs functional images,  usually resting or functional-tasks.
# This code may be unique, may be more efficient way to grab user code.
func_folders=glob.glob(os.path.join(data_path,'sub-*/*'))
# sort images
func_folders.sort()
#print(func_imgs)


# initialize fmri timeseries class object --inherites functional image list
obj1 = FMRITimeseries(func_folders)
# --test class
#print(obj1.data_dict)

# setup data dictionary with unique subject and task keys
data_dict=obj1.setup_dictionary(filtered_func=True)
#print(data_dict)


# set a list of subject ids from the dictionary 1st level keys
subject_ids=list(data_dict.keys())
subject_ids.sort()
subject_ct=len(subject_ids) # get count of subject dataset
print("[INFO] Dictionary made, {} keys \n[INFO] Keys: {}".format(len(data_dict.keys()), subject_ids))


# build chunklist
chunk_list=obj1.build_chunklist(subject_ids=subject_ids)
#print('[INFO] CPU ct: %s'%mp.cpu_count())


#########################################
## Step 2: Transform Functionals to match the atlas masks with flirt and fslmaths.
run_flirt=False
if run_flirt==True:
    print('[INFO] transforming functionals to match the mask with flirt....')

    flirt_start_time = time.time()
    for chunk in chunk_list:
        with mp.Pool(12) as p:
            #print(chunk)
            p.map(subject_loop, chunk)
    print('[INFO] transformation process complete.')
    flirt_process_time=time.time() - flirt_start_time
    print("--- %s seconds ---"%flirt_process_time )


run_fslmaths=True
if run_fslmaths==True:
    print('[INFO] apply new threhold with fslmaths,threshold 0.9...')

    fslmaths_start_time = time.time()
    for chunk in chunk_list:
        with mp.Pool(4) as p:
            #print(chunk)
            p.map(subject_loop, chunk)
    fslmaths_process_time=time.time() - fslmaths_start_time
    print("[INFO] process complete in --- %s seconds ---"%fslmaths_process_time )



#########################################
## Step 3: Pull individual ROI timeseries from each subject, by task/condition
